In [1]:
import os
import pandas as pd
import numpy as np
import scipy
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
os.getcwd()
os.chdir("C:\\Users\\Jacco Bruin .LAPTOP-ACFSBHIL\\THESIS")
df1 = pd.read_csv("6_year_dataset(4)(without_outliers).csv")    # change dataset to check result 3 year dataset and 1 year dataset 

df12 = df1["Ticker_x"]


df1 = df1.set_index("Ticker_x")

In [3]:
#scale data and add columns and index to it 
df11 = df1.columns
x = df1.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df1 = pd.DataFrame(x_scaled)
df1.columns = [df11]
df1.index = [df12]

In [4]:
selection_6_years = [1,2,3,4,5,7,8,9,10,11,12,15,16,17,20] #index number are here one less than in file "feature_selection" because different indicies
selection_3_years = [4,10,13,14,16,17]                           
selection_1_year = [1,2,3,4,5,6,7,8,9,10,11,12,15,16,17,24,27]

X = df1.iloc[:, 1:19]  # [:,1:29] = whole dataset, [:,1:19] = finanical ratios only, [:,19:29] = macroeconomic variables only
y = df1.iloc[:, -2]
X.columns

MultiIndex([(                      'EBITDA',),
            (                  'Total Debt',),
            (              'Free Cash Flow',),
            (         'Gross Profit Margin',),
            (            'Operating Margin',),
            (           'Net Profit Margin',),
            (            'Return on Equity',),
            (            'Return on Assets',),
            ('Free Cash Flow to Net Income',),
            (               'Current Ratio',),
            ( 'Liabilities to Equity Ratio',),
            (                  'Debt Ratio',),
            (   'Earnings Per Share, Basic',),
            ( 'Earnings Per Share, Diluted',),
            (             'Sales Per Share',),
            (            'Equity Per Share',),
            (    'Free Cash Flow Per Share',),
            (           'Pietroski F-Score',)],
           )

### Code to find the best hyperparameters

In [21]:
# RandomForestRegressor is used to optimalize the hyperparameters 

tree = RandomForestRegressor()
LR = {'max_depth': np.arange(1,11,2),          
      'n_estimators': np.arange(50,500,50),   
      'max_features': np.arange(5,20,3),        
      'bootstrap': ["True", "False"]     
     } 

clf = RandomizedSearchCV(tree, LR, cv = 10, scoring = "neg_mean_absolute_error", n_jobs = -1, random_state = 999 )
search = clf.fit(X, y)
search.best_params_, search.best_score_

C:\Users\Jacco Bruin .LAPTOP-ACFSBHIL\anaconda3\envs\tens\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the test scores are non-finite: [-0.20542482 -0.20934108 -0.20379524 -0.20379949         nan -0.20323625
 -0.20478376         nan -0.20334473 -0.20450263         nan -0.20256792
 -0.20508834 -0.20331943         nan -0.20337975 -0.20256721 -0.20514617
 -0.20342372 -0.2054082          nan         nan         nan -0.20250977
 -0.20621875 -0.20433059 -0.20446979         nan -0.20238102 -0.20296358
 -0.20311589         nan         nan -0.20289798 -0.20375493 -0.20321692
 -0.20294153 -0.20356791 -0.20317152 -0.20418323         nan -0.20296956
 -0.20244017         nan -0.20632819 -0.20424856 -0.20537894 -0.20396398
         nan         nan -0.20267362 -0.20340717 -0.2034892  -0.20342949
         nan -0.20463944 -0.20636441 -0.20301983         nan -0.20923358]
  category=UserWarning


({'n_estimators': 50, 'max_features': 8, 'max_depth': 5, 'bootstrap': 'False'},
 -0.2023810202814408)

## when is the best hyperparameters were found, the following code was used to find the results. 

In [7]:
selection_6_years = [1,2,3,4,5,7,8,9,10,11,12,15,16,17,20] #index number are here one less than in file "feature_selection" because different dataset     
selection_3_years = [4,10,13,14,16,17]                           
selection_1_year = [1,2,3,4,5,6,7,8,9,10,11,12,15,16,17,24,27]

X = df1.iloc[:, 1:29]  # [:,1:29] = whole dataset, [:,1:19] = finanical ratios only, [:,19:29] = macroeconomic variables only
y = df1.iloc[:, -2]
X.columns

MultiIndex([(                                               'EBITDA',),
            (                                           'Total Debt',),
            (                                       'Free Cash Flow',),
            (                                  'Gross Profit Margin',),
            (                                     'Operating Margin',),
            (                                    'Net Profit Margin',),
            (                                     'Return on Equity',),
            (                                     'Return on Assets',),
            (                         'Free Cash Flow to Net Income',),
            (                                        'Current Ratio',),
            (                          'Liabilities to Equity Ratio',),
            (                                           'Debt Ratio',),
            (                            'Earnings Per Share, Basic',),
            (                          'Earnings Per Share, Dilu

In [8]:
tree = RandomForestRegressor(n_estimators = 250, max_features = 8, max_depth = 5, bootstrap = False, random_state = 999)  # change the hyperparameters based on results randomsearchcv
result_cv = cross_validate(tree, X, y, cv = 5, scoring = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error"], return_train_score=True)  

print("-----------------")
#print("R2 of test Model with Cross Validation is:",result_cv["test_r2"].mean())
print("MSE of test Model with Cross Validation is:",result_cv['test_neg_mean_squared_error'].mean()*-1)
print("MAE of test Model with Cross Validation is:", result_cv['test_neg_mean_absolute_error'].mean()*-1)

### adjusted R2 score

# formula adjusted R2_score = 1 - (1-R2)*(len(y)-1)/(len(y)-X.shape[1]-1)

R2 = result_cv["test_r2"].mean()
n = len(y)
p = X.shape[1]

Adj_R2 = 1-(1-R2)*(n-1)/(n-p-1)

print("R2_adj of test Model with Cross Validation is:", Adj_R2)



### down the training result to check if the data was not overfitting

#print("------------------")
#print("R2 of train Model with Cross Validation is:",result_cv['train_r2'].mean())
#print("MSE of train Model with Cross Validation is:",result_cv['train_neg_mean_squared_error'].mean()*-1)
#print("MAE of train Model with Cross Validation is:", result_cv['train_neg_mean_absolute_error'].mean()*-1)

-----------------
MSE of test Model with Cross Validation is: 0.06524900065236763
MAE of test Model with Cross Validation is: 0.20044603804687994
R2_adj of test Model with Cross Validation is: 0.09561665160343613
